In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.layers import Dense

In [2]:
data = pd.read_csv("data/lerped-data.csv")

positions = data[["x", "y", "z"]].values.reshape(-1, 15, 3)
is_exploiting = data["is_exploiting"].values[::15]

print(positions.shape, is_exploiting.shape)

(1260, 15, 3) (1260,)


In [3]:
split = int(len(positions) * 0.8)

positions_train = positions[:split]
positions_test = positions[split:]

is_exploiting_train = is_exploiting[:split]
is_exploiting_test = is_exploiting[split:]

print(positions_train.shape, positions_test.shape, is_exploiting_train.shape, is_exploiting_test.shape)

(1008, 15, 3) (252, 15, 3) (1008,) (252,)


In [4]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(15, 3)),
    keras.layers.Dense(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam",
                loss="binary_crossentropy",
                metrics=["accuracy"])

model.fit(positions_train, is_exploiting_train, epochs=10, batch_size=10)

Epoch 1/10
101/101 [==============================] - 1s 3ms/step - loss: 1.9252 - accuracy: 0.7698
Epoch 2/10
101/101 [==============================] - 0s 3ms/step - loss: 0.6229 - accuracy: 0.8760
Epoch 3/10
101/101 [==============================] - 0s 3ms/step - loss: 0.4091 - accuracy: 0.9077
Epoch 4/10
101/101 [==============================] - 0s 3ms/step - loss: 0.5924 - accuracy: 0.8750
Epoch 5/10
101/101 [==============================] - 0s 3ms/step - loss: 0.4032 - accuracy: 0.9087
Epoch 6/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3356 - accuracy: 0.9296
Epoch 7/10
101/101 [==============================] - 0s 3ms/step - loss: 0.4440 - accuracy: 0.9077
Epoch 8/10
101/101 [==============================] - 0s 3ms/step - loss: 0.3794 - accuracy: 0.9236
Epoch 9/10
101/101 [==============================] - 0s 3ms/step - loss: 0.4925 - accuracy: 0.8919
Epoch 10/10
101/101 [==============================] - 0s 3ms/step - loss: 0.5844 - accuracy: 0.8859

In [5]:
model.evaluate(positions_test, is_exploiting_test)

8/8 [==============================] - 0s 3ms/step - loss: 5.7405 - accuracy: 0.4683


[5.740527153015137, 0.4682539701461792]

In [8]:
def parse(input_string, target_rows=15):
    positions = input_string.split("%")

    chunk_df = pd.DataFrame([list(map(float, pos.split(","))) for pos in positions], columns=["x", "y", "z"])

    if len(chunk_df) == target_rows:
        return chunk_df

    linspace = pd.DataFrame(
        index=pd.RangeIndex(target_rows),
        columns=chunk_df.columns
    )

    for col in ["x", "y", "z"]:
        linspace[col] = pd.Series(np.interp(
            np.linspace(0, 1, num=target_rows),
            np.linspace(0, 1, num=len(chunk_df)),
            chunk_df[col]
        ))

    return linspace.values.tolist()

In [31]:
data_string = "4.5,3.51,-3.75%4.5,3,-3.75%4.5,3,-3.75%3.31,8.19,25.51%2.82,10.34,37.65"

custom_test_positions = parse(data_string)
custom_test_positions = np.array(custom_test_positions)

prediction = model.predict(custom_test_positions.reshape(-1, 15, 3))[0][0]
print(f"raw prediction: {prediction}")
print(f"rounded: {np.round(prediction)}")

1/1 [==============================] - 0s 20ms/step
raw prediction: 0.9999988079071045
rounded: 1.0
